# **Machine to Machine (M2M) Querying [*MetadataFilters*](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter)**

### **What is M2M?**
EarthExplorer (EE) serves as a key data access portal, offering a range of tools for searching, discovering, and downloading data and metadata from the USGS Earth Resources Observation and Science (EROS) data repository. The Machine-to-Machine (M2M) API allows users to search and retrieve download URLS and metadata from the EROS archive by using programming languages like Python or PHP. Users can create JSON structures to pass to M2M endpoints, subsequently receiving JSON responses in return.

### **Requesting Access**
Users will need to log on to their [**EROS Registration Service (ERS)**](https://ers.cr.usgs.gov/) accounts to view view the M2M documentation. To submit download requests through M2M, ERS users must be authorized. 
Steps to request M2M access:
1. Login to [**EROS Registration Service (ERS)**](https://ers.cr.usgs.gov/) (See [**How to Create an ERS Account**](https://www.youtube.com/watch?v=Ut6kxbuP_nk))
2. Select the **Access Request** menu
3. From the **Access Controls** page, use the **Request Access** button
4. In the **Access Type** selector, choose **Access to EE's Machine to Machine interface (MACHINE)**, then complete the questions


<div class="alert alert-warning"><h4>
The USGS/EROS User Services team assesses these requests. Contact USGS EROS User Services for more information: <a href="https://www.usgs.gov/staff-profiles/usgs-eros-customer-services" target="_blank">custserv@usgs.gov</a> and visit the <a href="https://www.usgs.gov/centers/eros/science/earthexplorer-help-index#ers" target="_blank">EarthExplorer Help Index</a>   </h4></div>


### **Use Case Scenario**
This tutorial demonstrates how to query using metadata specific to the various collections of Landsat data. We use the following Machine-to-Machine API filters: [**metadata*Value***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue), [**metadata*Between***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween), [**metadata*And***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataAnd), and [**metadata*Or***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataOr).
<!-- The queryable metadata parameters vary depending on the dataset collection. For instance, Landsat 8-9 OLI/TIRS, Landsat 7 ETM+, and Landsat 4-5 TM may not have the same set of parameters available for querying. -->


Users can view queryable metadata after following one of these steps:
- Submitting a [**dataset-filters**](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) query and listing the results
- Submitting a search on [**Earth Explorer**](https://earthexplorer.usgs.gov/) and viewing the list of metadata available for querying under the **Additional Information** tab

The [**Landsat Collection 2 Data Dictionary**](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary) describes all queryable metadata available through Earth Explorer.

---
## **Setup**

#### Import necessary libraries

In [1]:
import json
import requests
from getpass import getpass
import sys
import time
import argparse
import cgi
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

C:\Users\gabriel.245\AppData\Local\Temp\3\ipykernel_40320\3098291382.py:7: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  import cgi


#### Define send request function
This function sends a request to a M2M endpoint and returns the parsed JSON response.

Input parameters include:
- **endpoint_url** (*str*): The URL of the M2M endpoint
- **payload** (*dict*): The payload to be sent with the request

Returns:
- **dict**: Parsed JSON response

In [2]:
# Send http request
def sendRequest(url, data, apiKey = None, exitIfNoResponse = True):
    """
    Send a request to an M2M endpoint and returns the parsed JSON response.

    Parameters:
    endpoint_url (str): The URL of the M2M endpoint
    payload (dict): The payload to be sent with the request

    Returns:
    dict: Parsed JSON response
    """  
    
    json_data = json.dumps(data)
    
    if apiKey == None:
        response = requests.post(url, json_data)
    else:
        headers = {'X-Auth-Token': apiKey}              
        response = requests.post(url, json_data, headers = headers)  
    
    try:
      httpStatusCode = response.status_code 
      if response == None:
          print("No output from service")
          if exitIfNoResponse: sys.exit()
          else: return False
      output = json.loads(response.text)
      if output['errorCode'] != None:
          print(output['errorCode'], "- ", output['errorMessage'])
          if exitIfNoResponse: sys.exit()
          else: return False
      if  httpStatusCode == 404:
          print("404 Not Found")
          if exitIfNoResponse: sys.exit()
          else: return False
      elif httpStatusCode == 401: 
          print("401 Unauthorized")
          if exitIfNoResponse: sys.exit()
          else: return False
      elif httpStatusCode == 400:
          print("Error Code", httpStatusCode)
          if exitIfNoResponse: sys.exit()
          else: return False
    except Exception as e: 
          response.close()
          print(e)
          if exitIfNoResponse: sys.exit()
          else: return False
    response.close()
    
    return output['data']

---
## **User Input**
#### **Login by entering [EROS Registration System (EROS)](https://ers.cr.usgs.gov/https://ers.cr.usgs.gov/) *username* and *password* when prompted**

In [ ]:
# print("Logging in...\n")

# serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"
# p = ['Enter EROS Registration System (ERS) Username: ', 'Enter ERS Account Password: ']

# # Use requests.post() to make the login request
# response = requests.post(f"{serviceUrl}login", json={'username': getpass(prompt=p[0]), 'password': getpass(prompt=p[1])})

# if response.status_code == 200:  # Check for successful response
#     apiKey = response.json()['data']
#     print('\nLogin Successful, API Key Received!')
#     headers = {'X-Auth-Token': apiKey}
# else:
#     print("\nLogin was unsuccessful, please try again or create an account at: https://ers.cr.usgs.gov/register.")

# # Print the API key only if login was successful
# if apiKey:
#     print("\nAPI Key: " + apiKey + "\n")

#### **OR**

#### **Login by setting [EROS Registration System (EROS)](https://ers.cr.usgs.gov/https://ers.cr.usgs.gov/) *username* and *password***

In [3]:
username = "gilisaacgabriel"
password = "winqiT-zecji8-tykjuf"

In [4]:
print("Logging in...\n")
    
serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"
login_payload = {'username' : username, 'password' : password}
    
apiKey = sendRequest(serviceUrl + "login", login_payload)
    
print("API Key: " + apiKey + "\n")

Logging in...

API Key: eyJjaWQiOjI3MTc0NzQzLCJzIjoiMTcyMDIxMjI0MyIsInIiOjQyMCwicCI6WyJ1c2VyIiwiZG93bmxvYWQiLCJvcmRlciJdfQ==



---
## **Setup General Search Payload for [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search)**
Below, we've setup a general search payload—a JSON  dictionary with various query parameters—to be transmitted to the M2M [**scene-search**](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) endpoint for querying products within the various collections of Landsat data. This payload includes:

- **maxResults:** the number of results to be returned (default is 100).

- **datasetName:** used to identify the collection of data to search.

- [**sceneFilters:**](https://m2m.cr.usgs.gov/api/docs/datatypes/#sceneFilter) used to filter the data within the collections of Landsat data. The ones used in this tutorial are:

    - The [**acquisitionFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#acquisitionFilter) uses the ***'start'*** and ***'end'*** dates in ISO format **yyyy-mm-dd** for the time period of interest. Due to variations in acquisition dates for each Landsat dataset, we won't set a fixed parameter. Instead, we'll wait to fill in the date range as appropriate for each dataset. An example setup is:
    `acquisitionFilter = {'start' : '2023-03-01', 'end' : '2023-04-01'}`

    - The [**cloudCoverFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#cloudCoverFilter) is used to filter by percent cloud cover by setting a ***'min'*** and ***'max'*** (**%**).
    
    - The [**metadataFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter), which is the focus of this tutorial, includes the [**metadataValue**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue), [**metadataAnd**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataAnd), [**metadataOr**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataOr), and [**metadataBetween**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween). Each of these will be explored in detail in the sections that follow.


### **Create a function that creates general search payload**
In this section, we define a search payload function that generates a JSON dictionary which will be used for running a [**scene-search**](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search). The function takes input parameters such as **datasetName**, [**metadataFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter), and [**acquisitionFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#acquisitionFilter). Default parameters like **maxResults** (set to 10) and [**cloudCoverFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#cloudCoverFilter) (set to 50%) are also included.

In general, this function simplifies the process of generating a JSON dictionary while applying metadataFilters to various Landsat dataset collections.

In [ ]:
def create_search_payload(datasetName, metadataFilter, acquisitionFilter):
    # create search payload from input filters
    search_payload = {
        "maxResults": 10,
        'datasetName' : datasetName,
        'sceneFilter' : {
            'metadataFilter': metadataFilter,
            'acquisitionFilter' : acquisitionFilter,
            'cloudCoverFilter' : {'min' : 0, 'max' : 50},
        }
    }
    return search_payload

<div class="alert alert-info" role="alert">
  <h4>
    View the list of Landsat <a href="./M2M_Landsat_Collections_datasetNames.html" target="_blank">&nbsp;datasetNames</a> available for querying the M2M API.
  </h4>
</div>

# [**1. metadata*Value***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue)

---
## [**Searching for a *Satellite***](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#satellite)

To show how to use the [**metadataValue**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue) we to search for a specific satellite using the [**Landsat 5 data from the Landsat 4-5 Thematic Mapper Collection 2 Level-1**](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-landsat-archives-landsat-4-5-thematic-mapper-collection-2) (***landsat_tm_c2_l1***) dataset.
We start by sending a [**dataset-filters**](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) request to retrieve the **`'id'`** for the field we are filtering for, in this case it is **`'Satellite'`**. This **`'id'`** will be used in the metadataFilter parameter **`'filterId'`**. The [**dataset-filters**](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) request returns the [**metadataFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter) parameters for the specified dataset. These values can be used as additional criteria when submitting search queries for specific scenes. The **'datasetName'** input parameter is necessary. 
#### **1. Run [dataset-filters](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters)**

In [ ]:
# datasetfilters_tmc2L1 = sendRequest(serviceUrl + "dataset-filters", {'datasetName': 'landsat_tm_c2_l1'}, apiKey)

In [5]:
datasetfilters_tmc2L1 = sendRequest(serviceUrl + "dataset-filters", {'datasetName': 'gls2000_dem'}, apiKey)

#### **2. Use Pandas to view the [dataset-filters](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) result**
Below we show the results from the [**dataset-filters**](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) query as a table using `pandas`. The **`fieldLabel`** collumn provides insights into the queryable fields available for this specific Landsat dataset. The fields in these results are needed for figuring out which filter options are available, and applying them. While the **`fieldConfig.type`** lists one of three types for each fields:  **`Text`**, **`Range`** or **`Select`**. These act as a guide on the appropriate [**metadataFilter**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataFilter) type to apply. 
  
If the `fieldConfig.type` (also `'fieldConfig':'type'`) is:
- `Select`, then check the `valueList` field for valid values to use in a `'filterType':'value'` metadataFilter ([metadataValue](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue))
- `Range`, then use a `'filterType':'between'` metadataFilter and provide `'firstValue'` and `'secondValue'` ([metadataBetween](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween))
- `Text`, then use a `'like'` or `'='` operand in a `'filterType':'value'` metadataFilter ([metadataValue](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue))


In [6]:
pd.json_normalize(datasetfilters_tmc2L1)

,id,legacyFieldId,dictionaryLink,fieldLabel,searchSql,fieldConfig.type,fieldConfig.filters,fieldConfig.validators,fieldConfig.displayListId
0,63dd4574b733d055,None,https://www.usgs.gov/centers/eros/science/land...,ULLAT,ul_corner_lat between ?,Range,[],[],
1,63dd4574d7b638fb,None,https://www.usgs.gov/centers/eros/science/land...,ULLON,ul_corner_lon between ?,Range,[],[],
2,63dd457415727274,None,https://www.usgs.gov/centers/eros/science/land...,ULXMAP,ulxmap between ?,Range,[],[],
3,63dd45741aab80d5,None,https://www.usgs.gov/centers/eros/science/land...,ULYMAP,ulymap between ?,Range,[],[],


#### **3. Identify the index for the filter parameter, `fieldLabel : Satellite`**
In this case it is index # 3. We will use the **`id`** for the **`filterId`** in the [metadataValue](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue) dictionary below. 
When the **`'fieldConfig':'type'`** is **`Select`**, users are encouraged to check the **`valueList`** field for valid values to use in a metadataValue filter.

The **`valueList`** shows the querable satellite values for this collection of Landsat data they are **`'All'`** for all satellites,  **`'5'`** for Landsat 5 and **`'4'`** for Landsat 4.

In [11]:
datasetfilters_tmc2L1[0]

{'id': '63dd4574b733d055',
 'legacyFieldId': None,
 'dictionaryLink': 'https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal',
 'fieldConfig': {'type': 'Range',
  'filters': [],
  'options': {},
  'validators': [],
  'displayListId': ''},
 'fieldLabel': 'ULLAT',
 'searchSql': 'ul_corner_lat between ?'}

#### **4. Fill `metadataFilter` parameter with information for querying Landsat-5 satellites**
- The `'filterType'` is `'value'` since the `'fieldConfig':'type'` is `Select` for the `'fieldLabel': 'Satellite'`
- The `'filterId': datasetfilters_tmc2L1[3]['id']` points to the `'id'` value for querying satellite data within the Landsat 4-5 Thematic Mapper Collection 2 Level-1 (`'landsat_tm_c2_l1'`) dataset.
- The `'value' : '5'` is set to query for Landsat 5 scenes within the Landsat 4-5 Thematic Mapper Collection 2 Level-1 (`'landsat_tm_c2_l1'`) dataset.

In [12]:
satellite_filter = {'filterType': 'value',
                 'filterId': datasetfilters_tmc2L1[0]['id'],
                 'value' : '5'}
satellite_filter

{'filterType': 'value', 'filterId': '63dd4574b733d055', 'value': '5'}

#### **5. Create the [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) payload for querying Landsat-5 products from the Landsat 4-5 Thematic Mapper Collection 2 Level-1**

Note that the Landsat 4-5 Thematic Mapper dataset was acquired between March 1984 and January 2013 ([**Landsat Missions Timeline**](https://www.usgs.gov/media/images/landsat-missions-timeline)), therefore the **aquisitionFilter** dates should be within those ranges.

In [13]:
satellite_payload = create_search_payload('gls2000_dem', satellite_filter, {})
satellite_payload
satellite_payload

NameError: name 'create_search_payload' is not defined

#### **6. Run [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search)  and view results** 
The code below runs the [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) endpoint and shows the results using pandas. Note the returned results are all Landsat-5 scenes indicated by the **`'LT05'`** in the **`displayId`** names (see the [**Landsat Product ID**](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary/#landsat_product_id) to learn about the naming conventions).

In [8]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", satellite_payload, apiKey))['results'])

NameError: name 'satellite_payload' is not defined

## [**Landsat Scene Identifier**](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary/#landsat_scene_id)

Here we continue to show the uses of the metadataValue filter by searching for products with a specific [**Landsat Scene Identifier**](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary/#landsat_scene_id) in the same Landsat 4-5 Thematic Mapper Collection 2 Level-1 dataset. The [**Landsat Scene Identifier**](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary/#landsat_scene_id), also known as the **entityID**, is a unique identifier assigned to each Landsat scene. It is part of the metadata associated with Landsat scenes and serves as a specific identifier for a particular observation or image taken by a Landsat satellite. The **entityId** is a string composed of various components that provide information about the Landsat scene, such as the satellite platform, sensor, date of acquisition, and path/row. We will be querying for data within a scene with the **entityId**: **`LT50380322011251PAC02`**.

#### **1. Identify the index for the filter parameter, `fieldLabel : Landsat Scene Identifier`**
In this case it is index # 11. 

In [14]:
datasetfilters_tmc2L1[11]

IndexError: list index out of range

#### **2. Fill `metadataFilter` parameter with information for querying a specific Landsat Scene Identifier**
- The `'filterType'` is `'value'` since the `'fieldConfig':'type'` is `Text` for the `'fieldLabel': 'Landsat Scene Identifier'`
- The `'filterId': datasetfilters_tmc2L1[11]['id']` points to the `'id'` value for querying  Landsat Scene Identifier data within the Landsat 4-5 Thematic Mapper Collection 2 Level-1 (`'landsat_tm_c2_l1'`) dataset.
- The `'value' : 'LT50380322011251PAC02'` is set to query for data within a Landsat 4-5 Thematic Mapper Collection 2 Level-1 (`'landsat_tm_c2_l1'`) scene, where *'LT50380322011251PAC02'* is the entityId for the scene
- The `'operand'` is `'='` to find scenes within the specific Scene Identifier

In [ ]:
scene_identifier_filter = {'filterType': 'value',
                 'filterId': datasetfilters_tmc2L1[11]['id'],
                 'value' : 'LT50380322011251PAC02',
                           'operand' : '='}
scene_identifier_filter

#### **3. Create the [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) payload used for finding products within the `LT50380322011251PAC02` scene**

In [ ]:
scene_identifier_payload = create_search_payload('landsat_tm_c2_l1', scene_identifier_filter, {'start' : '1994-01-01', 'end' : '2013-01-31'})
scene_identifier_payload

#### **4. Run [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search)  and view results** 
The result shows a single scene with the Scene Identifier `LT50380322011251PAC02`, this is indicated by the `entityId` names.

In [ ]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", scene_identifier_payload, apiKey))['results'])

---
## [**Tile Grid Region**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_grid_region)
Another way to use the [**metadataValue**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue) filter is for specifying [**Tile Grid Region**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_grid_region) in the Analysis Ready Data (ARD) dataset. The [**Tile Grid Region**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_grid_region) is also specific to Level-3 Landsat 4-9 Collection 2 (C2) Datasets (e.g. [**Dynamic Surface Water Extent (DWSE)**](https://www.usgs.gov/landsat-missions/landsat-collection-2-level-3-dynamic-surface-water-extent-science-product), [**Burned Area**](https://www.usgs.gov/landsat-missions/landsat-collection-2-level-3-burned-area-science-product), [**Fractional Snow Covered Area**](https://www.usgs.gov/landsat-missions/landsat-collection-2-level-3-fractional-snow-covered-area-science-product), [**Fractional Snow Covered Area (fSCA) Statistics**](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-landsat-collection-2-level-3-fsca-statistics-science-product)) which are processed from [**Landsat Collection 2 (C2) U.S. Analysis Ready Data (ARD)**](https://www.usgs.gov/landsat-missions/landsat-collection-2-us-analysis-ready-data).

Our example below uses the [**Landsat 4-9 Collection 2 (C2) U.S. Analysis Ready Data (ARD) Level-2 Tiles (Albers projection)**](https://www.usgs.gov/landsat-missions/landsat-collection-2-us-analysis-ready-data) (***landsat_ard_tile_c2***) to query for tiles in **Alaska**.

#### **1. Run [dataset-filter](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-filters) endpoint for *`landsat_ard_tile_c2`* and use pandas to view**

In [ ]:
datasetfilters_ardtilec2 = sendRequest(serviceUrl + "dataset-filters", {'datasetName': 'landsat_ard_tile_c2'}, apiKey)
pd.json_normalize(datasetfilters_ardtilec2)

#### **2. Identify the index for the filter parameter, `fieldLabel : Tile Grid Region`**
In this case it is index # 0. The **`valueList`** shows the querable Tile Grid Regions which are `'All'` for all regions,  `'CU'` for conterminous United States, `'AK'` for Alaska and `'HI'` for Hawaii.

In [ ]:
datasetfilters_ardtilec2[0]

#### **3. Fill `metadataFilter` parameter for querying the Tile Grid Region**
- The `'filterType'` is `'value'` since the `'fieldConfig':'type'` is `Select` for the `fieldLabel : Tile Grid Region`
- The `'filterId': datasetfilters_ardtilec2[0]['id']` points to the `'id'` value for querying a specific Tile Grid Region within the Landsat 4-9 Collection 2 (C2) U.S. Analysis Ready Data (ARD) Level-2 Tiles (`'landsat_ard_tile_c2'`) dataset.
- The `'value' : 'AK'` is set to query for tiles in Alaska

In [ ]:
tileregion_filter = {'filterId': datasetfilters_ardtilec2[0]['id'],
                 'filterType': 'value',
                 'value': 'AK'}
tileregion_filter

#### **4. Create [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) payload for retrieving data from a specific Tile Grid Region**

In [ ]:
tile_region_payload = create_search_payload('landsat_ard_tile_c2', tileregion_filter, {'start' : '2009-09-01', 'end' : '2009-09-05'})
tile_region_payload

#### **5. Run [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search)  and view results** 
The code below runs the [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) endpoint and shows the results using pandas. Note the returned results are all Alaska scenes indicated by the **'AK'** in the **`entityId`** and **`displayId`** names (see the [**Landsat Tile Identifier**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_identifier)).

In [ ]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", tile_region_payload, apiKey))['results'])

<div class="alert alert-info"><h4>To understand the tile naming convention, please refer to the <a href="https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_identifier" target="_blank"><b><i>Tile Identifier </i></b></a>documentation.</h4></div>

# [**metadata*Between***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween)

The [**metadataBetween**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween) is used to find products within a specified **`Range`**, i.e. values or dates, depending on the metadata being queried. A reminder that if the **`fieldConfig.tile`** is **`Range`**, a  **`firstValue`** and **`secondValue`** are necessary.

---
## [**Tile Grid Horizontal**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_grid_horizontal)
Here we use the [**metadataBetween**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween) filter to find products within a range of [**Tile Grid Horizontal**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_grid_horizontal) with the same [**Landsat 4-9 Collection 2 (C2) U.S. Analysis Ready Data (ARD) Level-2 Tiles (Albers projection)**](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-landsat-archives-landsat-collection-2-us-analysis-ready-data) (***landsat_ard_tile_c2***) dataset used above.

#### **1. Identify the index for the filter parameter, `fieldLabel : Tile Grid Horizontal`**
In this case it is index # 1. We will use the **`id`** for the **`filterId`** in the metadataBetween dictionary below. 

In [ ]:
datasetfilters_ardtilec2[1]

#### **2. Fill `metadataFilter` parameter**
- The `'filterType'` is `'between'` since the `'fieldConfig':'type'` is `Range` for the `'fieldLabel': 'Tile Grid Horizontal'`
- The `'filterId': datasetfilters_ardtilec2[1]['id']` points to the `'id'` value for querying Tile Grid Horizontal ranges within this particular dataset
- The `'firstValue' : '8'` is the start of the range and `'secondValue': 10` is the end of the range

In [ ]:
tilehorizontal_filter = {'filterType': 'between',
                    'filterId': datasetfilters_ardtilec2[1]['id'],
                    'firstValue': 8,
                    'secondValue': 10}
tilehorizontal_filter

#### **3. Create and view search payload for the Tile Grid Horizontal payload**

In [ ]:
tile_horizontal_payload = create_search_payload('landsat_ard_tile_c2', tilehorizontal_filter, {'start' : '2023-01-01', 'end' : '2023-11-01'})
tile_horizontal_payload

#### **4. Run [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search)  and view results**
The results show the **HHH** (Horizontal Tile Number) in both the **`entityIds`** and **`displayIds`** ranging from **`'008'`** to **`'010'`**, for an ARD with a naming convention of **`LXSS_US_HHHVVV_YYYYMMDD_yyyymmdd_CCC_VVV`** (see the ARD naming conventions in the [**Tile Identifier**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary/#tile_identifier)).

In [ ]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", tile_horizontal_payload, apiKey))['results'])

---
## [**Tile Production Date**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_production_date)
We can also filter using the production date **`Tile Production Date (Ex. YYYY/MM/DD)`** metadataFilter using the [**Landsat 4-9 Collection 2 (C2) U.S. Analysis Ready Data (ARD) Level-2 Tiles (Albers projection)**](https://www.usgs.gov/landsat-missions/landsat-collection-2-us-analysis-ready-data) (***landsat_ard_tile_c2***). Here, we limit the search to production dates between **July-1-2023** and **July-31-2023**.

#### **1. Identify the index for the filter parameter, `fieldLabel : Tile Production Date (Ex. YYYY/MM/DD)`**
In this case it is index # 4. We will use the **`id`** for the **`filterId`** in the metadataBetween dictionary below. 

In [ ]:
datasetfilters_ardtilec2[4]

#### **2. Fill Tile Production Date `metadataFilter` parameter**
- The `'filterType'` is `'between'` since the `'fieldConfig':'type'` is `Range` for the `'fieldLabel': 'Tile Production Date (Ex. YYYY/MM/DD)'`
- The `'filterId': datasetfilters_ardtilec2[4]['id']` points to the `'id'` value for querying Tile Production Date ranges within this particular dataset
- The `'firstValue' : '2023/07/01'` is the start of the date range and `'secondValue': 2023/07/31` is the end of the date range

In [ ]:
tileproddate_filter = {'filterId': datasetfilters_ardtilec2[4]['id'],
                   'filterType': 'between',
                   'firstValue': '2023/07/01',
                   'secondValue': '2023/07/31'}
tileproddate_filter

#### **3. Create and View Tile Production Date Search Payload**

Using the similar acquisitionFilter dates as above `{'start' : '2023-01-01', 'end' : '2023-11-01'}` we expect results to be narrowed to the Tile Production Date dates in the new `tileproddate_betweenfilter` paramater i.e. between `'2023/07/01'` and `'2023/07/31'`.

In [ ]:
tile_prod_date_payload = create_search_payload('landsat_ard_tile_c2', tileproddate_filter, {'start' : '2023-01-01', 'end' : '2023-11-01'})
tile_prod_date_payload

#### **4. Run and view the [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) payload results for the Tile Production Date Search**

The results from the query show products within the Tile Production Date where the **`yyyymmdd`** in the [**ARD Tile Identifier**](https://www.usgs.gov/centers/eros/science/ard-landsat-tiles-data-dictionary#tile_identifier) **`LXSS_US_HHHVVV_YYYYMMDD_yyyymmdd_CCC_VVV`** is between the specified production dates **`'2023/07/20'`** and **`'2023/07/31'`**.

In [ ]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", tile_prod_date_payload, apiKey))['results'])

---
# [**metadata*And***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataAnd)
The [**metadataAnd**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataAnd) is used to make combinations of the [**metadataValue**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue) and/or [**metadataBetween**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween)
 filter types.
 For this example we query within the **Landsat 8-9 Operational Land Imager and Thermal Infrared Sensor Collection 2 Level-2 (*landsat_ot_c2_l2*)** dataset.

In [ ]:
datasetfilters_otc2l2 = sendRequest(serviceUrl + "dataset-filters", {'datasetName': 'landsat_ot_c2_l2'}, apiKey)
pd.json_normalize(datasetfilters_otc2l2)

---
## **Combine [**metadata*Between***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween) filters with [metadata*And*](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataOr)**

We can start by finding data within specific ranges of the **`WRS Path`** and **`WRS Row`**.

#### **1. Create and print the [WRS Path](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row) *between* filter**
The WRS Path is at index # 3.

In [ ]:
datasetfilters_otc2l2[3]

- The `'filterType'` is `'between'` since the `'fieldConfig':'type'` is `Range` for the `'fieldLabel': 'WRS Path'`
- The `'filterId': datasetfilters_otc2l2[3]['id']` points to the `'id'` value for querying WRS Path ranges within this particular dataset
- The `'firstValue' : 29` is the start and `'secondValue': 31` is the end of the WRS path range

In [ ]:
WRSpath_filter = {'filterId': datasetfilters_otc2l2[3]['id'],
                   'filterType': 'between',
                   'firstValue': 29,
                   'secondValue': 31}
WRSpath_filter

#### **2. Create and print the [WRS Row](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row) *between* filter**
The WRS Path is at index # 4.

In [ ]:
datasetfilters_otc2l2[4]

- The `'filterType'` is `'between'` since the `'fieldConfig':'type'` is `Range` for the `'fieldLabel': 'WRS Row'`
- The `'filterId': datasetfilters_otc2l2[4]['id']` points to the `'id'` value for querying WRS Row ranges within this particular dataset
- The `'firstValue' : 29` is the start and `'secondValue': 31` is the end of the WRS Row range

In [ ]:
WRSrow_filter = {'filterId': datasetfilters_otc2l2[4]['id'],
                   'filterType': 'between',
                   'firstValue': 29,
                   'secondValue': 31}
WRSrow_filter

#### **3. Combine into a [**metadataAnd**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataAnd) filter parameter**
- The `'filterType'` is `'and'` since we are combining two `'between'` metadataFilters
- The `'childFilters'` are  the `WRSpath_filter` and the `WRSrow_filter`, `'between'` filters created above

In [ ]:
WRS_path_row_andfilter = {'filterType': 'and',
              'childFilters':[WRSpath_filter, WRSrow_filter]}

WRS_path_row_andfilter

#### **4. Create and View WRS Path and Row Search Payload**
We expect results to show dataset between PATHs and ROWs Grid `29` and `31`.

In [ ]:
WRS_path_row_payload = create_search_payload('landsat_ot_c2_l2', WRS_path_row_andfilter, {'start' : '2023-01-01', 'end' : '2023-11-01'})
WRS_path_row_payload

#### **5. Run and view the [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) payload results for WRS Path and Row Search**

The results from the query show **`displayIDs`** with **`PPP`** (WRS Path number) and **`RRR`** (WRS ROw numbers) from **`29`** to **`31`** (see the [**Landsat Collection 2 Product Identifier**](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary/#landsat_product_id) **`LXSS_LLLL_PPPRRR_YYYYMMDD_yyyymmdd_CC_TX`**). 

In [ ]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", WRS_path_row_payload, apiKey))['results'])

---
## **Combine a [**metadata*Value***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue) and [**metadata*Between***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween) filter with [metadata*And*](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataOr)**

We can combine both **`WRSpath_filter`** and **`WRSrow_filter`** **`'between'`** filters created above with a **Collection Category** **`'value'`** filter.

#### **1. Create and print the [Collection Category](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_category) *value* filter**
The index is 8 for the Collection Category. 

In [ ]:
datasetfilters_otc2l2[8]

The **`valueList`** shows the querable Collection Category which are **`'All'`** for all tiers of the processed data: **`'T1'`** or **`'Tier 1'`** is the highest available quality and processing level, **`'T2'`** or **`'Tier 2'`** for data not to the level of Tier 1 (i.e. significant significant cloud cover) **`'RT'`** or **`'Real-Time'`** for data that requires additional processing to be Tier 1 and Tier 2.

In [ ]:
collectioncategory_filter = {'filterType': 'value',
             'filterId': datasetfilters_otc2l2[8]['id'],
             'value' : 'T2'}
collectioncategory_filter

#### **2. Combine into a [**metadataAnd**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataAnd) filter parameter**

- The `'filterType'` is `'and'` since we are combining the `'between'` and `'value'` filters
- The `'childFilters'` are  the `WRSpath_filter`, `WRSrow_filter`, the `'between'` filters created above and the new `collectioncategory_filter` `'value'` filter

In [ ]:
wrspathrow_category_andfilter = {'filterType': 'and',
              'childFilters':[WRSpath_filter, WRSrow_filter, collectioncategory_filter]}

wrspathrow_category_andfilter

#### **3. Create and View WRS Path, WRS Row and Collection Category Search Payload**
We expect results to show dataset between PATHs and ROWs `29` and `31` with only Tier 2 `'T2'` level quality.

In [ ]:
WRS_path_row_category_payload = create_search_payload('landsat_ot_c2_l2', wrspathrow_category_andfilter, {'start' : '2023-01-01', 'end' : '2023-11-01'})
WRS_path_row_category_payload

#### **4. Run and view the [*scene-search*](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) payload results for WRS Path, WRS Row and Collection Category Search**

The results from the query show **`displayID's`** with **`PPP`** (WRS Path number), **`RRR`** (WRS Row numbers) between **`29`** and **`31`**, and **`TX`** (Collection Category) **`T2`**.(see the [**Landsat 8-9 Collection 2 Level 1 Naming Convention**](https://www.usgs.gov/faqs/what-naming-convention-landsat-collections-level-1-scenes) **`LXSS_LLLL_PPPRRR_YYYYMMDD_yyyymmdd_CC_TX`**). 

In [ ]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", WRS_path_row_category_payload, apiKey))['results'])

---
# [**metadata*Or***](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataOr)
Like the [**metadataAnd**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataAnd), the [**metadataOr**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataOr) is used to make combininations of the [**metadataValue**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataValue) and/or [**metadataBetween**](https://m2m.cr.usgs.gov/api/docs/datatypes/#metadataBetween) filter types.

#### **1. Create and print the [Landsat Scene Identifier](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary/#landsat_scene_id) *value* filter**
The index is 2 for the Landsat Scene Identifier.

In [ ]:
datasetfilters_otc2l2[2]

#### **2. Fill `metadataFilter` parameter with information for querying a specific `Landsat Scene Identifier`**
- The `'filterType'` is `'value'` since the `'fieldConfig':'type'` is `Text` for the `'fieldLabel': 'Landsat Scene Identifier'`
- The `'filterId': datasetfilters_tmc2L1[11]['id']` points to the `'id'` value for querying the Landsat 8-9 Operational Land Imager and Thermal Infrared Sensor Collection 2 Level-2 (`'landsat_ot_c2_l2'`) dataset
- The `'value' : 'LC80460272023260LGN00'`, `'value' : 'LC90470272023259LGN00'` and `'value' : 'LC90470272023195LGN00'` is set to query for data within a specific Landsat 8-9 Operational Land Imager and Thermal Infrared Sensor Collection 2 Level-2  (`'landsat_ot_c2_l2'`) dataset scene
- The `'operand'` is `'like'` to find scenes within the specific Scene Identifier

In [ ]:
scene_identifier_orfilter = {'filterType': 'or',
              'childFilters':[
                  {'filterType': 'value',
                 'filterId': datasetfilters_tmc2L1[11]['id'],
                 'value' : 'LC80460272023260LGN00',
                           'operand' : 'like'},
                  {'filterType': 'value',
                 'filterId': datasetfilters_tmc2L1[11]['id'],
                 'value' : 'LC90470272023259LGN00',
                           'operand' : 'like'},
                  {'filterType': 'value',
                 'filterId': datasetfilters_tmc2L1[11]['id'],
                 'value' : 'LC90470272023195LGN00',
                           'operand' : 'like'}]}
scene_identifier_orfilter

#### **3. Create the [scene-search](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) payload used for querying the `LC80460272023260LGN00`, `LC90470272023259LGN00` and `LC90470272023195LGN00` scenes**

In [ ]:
ot_scene_identifier_payload = create_search_payload('landsat_ot_c2_l2', scene_identifier_orfilter, {'start' : '2023-01-01', 'end' : '2023-11-01'})
ot_scene_identifier_payload

#### **4. Run [*scene-search*](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search)  and view results** 
The result shows scenes with the Scene Identifiers **`LC80460272023260LGN00`**, **`LC90470272023259LGN00`** and **`LC90470272023195LGN00`**, this is indicated by the **`entityId`** names.

In [ ]:
pd.json_normalize((sendRequest(serviceUrl + "scene-search", ot_scene_identifier_payload, apiKey))['results'])

---
## **[Logout](https://m2m.cr.usgs.gov/api/docs/reference/#logout)**<a id="logout-api"></a>

Logout so the API Key cannot be used anymore.

<div class="alert alert-block alert-warning">
    <h4> <b>NOTE:</b> The Machine-to-Machine API key expires after 2 hours of inactivity. </h4>
</div>  

In [ ]:
endpoint = "logout"  
if sendRequest(serviceUrl + endpoint, None, apiKey) == None:        
    print("\n\nLogged Out\n")
else:
    print("\n\nLogout Failed\n")

---
<div class="alert alert-block alert-info">
    <h1> Contact Information </h1>
    <h3> Material written by Tonian Robinson<sup>1</sup> </h3>
    <ul>
        <b>Contact:</b> custserv@usgs.gov <br> 
        <b>Voice:</b> +1-605-594-6151 <br>
        <b>Organization:</b> USGS EROS User Services <br>
        <b>Date last modified:</b> 24-Jan-2024 <br>
    </ul>
    
<sup>1</sup>Earth Space Technology Services LLC., contractor to the U.S. Geological Survey, Earth Resources Observation and Science (EROS) Center, Sioux Falls, South Dakota, 57198-001, USA. Work performed under USGS contract G0121D0001.
</div>